In [17]:
import numpy as np
from pathlib import Path
import shutil

import spikeinterface as si
from spikeinterface.exporters import export_to_phy

In [23]:
session_folder = Path('/data/ecephys_661398_2023-03-31_17-01-09')
sorting_folder = Path('/data/ecephys_661398_2023-03-31_17-01-09_sorted-ks2.5')
scratch_folder = Path('/scratch')
results_folder = Path('/results')
ecephys_compressed_folder = session_folder / 'ecephys_compressed'
sorting_curated_folder = sorting_folder / "sorting_precurated"
postprocessed_folder = sorting_folder / 'postprocessed'
ap_stream_name = 'experiment1_Record Node 104#Neuropix-PXI-100.ProbeA-AP'
lfp_stream_name = 'experiment1_Record Node 104#Neuropix-PXI-100.ProbeA-LFP'
dat_path = session_folder / 'ephys_clipped' / 'Record Node 104' / 'experiment1' / 'recording1' / 'continuous' / 'Neuropix-PXI-100.ProbeA-AP' / 'continuous.dat' 

In [5]:
we_recless = si.load_waveforms(postprocessed_folder / f'{ap_stream_name}_recording1', with_recording=False)

/opt/conda/lib/python3.9/site-packages/spikeinterface/core/base.py:998: UserWarning: Versions are not the same. This might lead compatibility errors. Using spikeinterface==0.97.1 is recommended
  warnings.warn(


In [6]:
we_recless.channel_ids[:5]

array(['AP1', 'AP2', 'AP3', 'AP4', 'AP5'], dtype='<U5')

In [7]:

we_recless.get_channel_locations()[:5]

array([[11.,  0.],
       [59.,  0.],
       [11., 20.],
       [59., 20.],
       [11., 40.]])

In [3]:
tmp_we_path = scratch_folder / postprocessed_folder.parent.name / "postprocessed" / ap_stream_name

In [16]:

shutil.copytree(we_recless.folder, tmp_we_path)

PosixPath('/scratch/ecephys_661398_2023-03-31_17-01-09_sorted-ks2.5/postprocessed/experiment1_Record Node 104#Neuropix-PXI-100.ProbeA-AP')

In [8]:
we_recless_local = si.load_waveforms(tmp_we_path, with_recording=False,
                                     sorting=we_recless.sorting)
print(we_recless_local)

WaveformExtractor: 358 channels - 289 units - 1 segments
  before:90 after:120 n_per_units:500 - sparse


In [9]:
phy_folder = results_folder / f"{postprocessed_folder.parent.name}_phy"

In [ ]:
we_recless_local.

In [14]:
export_to_phy(we_recless_local, 
                   output_folder=phy_folder,
                   compute_pc_features=False,
                   remove_if_exists=True,
                   copy_binary=False)

/opt/conda/lib/python3.9/site-packages/spikeinterface/exporters/to_phy.py:85: UserWarning: Exporting to Phy with many channels and without sparsity might result in a heavy and less informative visualization. You can use use a sparse WaveformExtractor or you can use the 'sparsity' argument to enforce sparsity (see compute_sparsity())
  warnings.warn(


Run:
phy template-gui  /results/ecephys_661398_2023-03-31_17-01-09_sorted-ks2.5_phy/params.py


In [15]:
spike_locations = we_recless_local.load_extension("spike_locations").get_data()
spike_depths = spike_locations["y"]

In [18]:
np.save(phy_folder / "spike.depths.npy", spike_depths)

In [21]:
from phylib.io import model
from phylib.io import alf

In [53]:
m = phy_model_from_ks2_path(phy_folder, dat_path, 30000, 384)

In [52]:
out_path = '/results/alf'

ac = alf.EphysAlfCreator(m)
ac.convert(out_path) #, label=label, force=force, ampfactor=ampfactor)

# set depths to spike_depths to catch cases where it can't be computed from pc features (e.g in case of KS3)
#m.depths = np.load(out_path.joinpath('spikes.depths.npy'))
#ephysqc.spike_sorting_metrics_ks2(ks_path, m, save=True, save_path=out_path)

Converting to ALF:  20%|██        | 25/125 [00:00<00:00, 125.96it/s]


AttributeError: 'int' object has no attribute 'sparse_templates'

In [41]:
m.sparse_templates.cols

array([[  0,   1,   2, ...,  -1,  -1,  -1],
       [  0,   1,   2, ...,  17,  18,  -1],
       [  0,   1,   2, ...,  17,  18,  -1],
       ...,
       [319, 321, 322, ..., 337, 338, 339],
       [310, 311, 312, ..., 327, 328, 330],
       [317, 319, 320, ..., 335, 336, 337]])

In [29]:
NCH_WAVEFORMS = 32  # number of channels to be saved in templates.waveforms and channels.waveforms

def phy_model_from_ks2_path(ks2_path, dat_path, sample_rate, n_channels_dat):
    
    m = model.TemplateModel(dir_path=ks2_path,
                            dat_path=dat_path,  # this assumes the raw data is in the same folder
                            sample_rate=sample_rate,
                            n_channels_dat=n_channels_dat,
                            n_closest_channels=NCH_WAVEFORMS)
    m.depths = m.get_depths()
    return m

In [50]:
def get_amplitudes_true(self, sample22unit=1., use='templates'):
    """Convert spike amplitude values to input amplitudes units
     via scaling by unwhitened template waveform.
     :param sample2unit float: factor to convert the raw data to a physical unit (defaults 1.)
     :returns: spike_amplitudes_volts: np.array [nspikes] spike amplitudes in raw data units
     :returns: templates_volts: np.array[ntemplates, nsamples, nchannels]: templates
     in raw data units
     :returns: template_amps_volts: np.array[ntemplates]: average templates amplitudes
      in raw data units
     To scale the template for template matching,
     raw_data_volts = templates_volts * spike_amplitudes_volts / template_amps_volts
     """
    # spike_amp = ks2_spike_amps * maxmin(inv_whitening(ks2_template_amps))
    # to rescale the template,

    if use == 'clusters':
        sparse = self.sparse_clusters
        spikes = self.spike_clusters
        n_wav = self.n_clusters
    else:
        sparse = self.sparse_templates
        spikes = self.spike_templates
        n_wav = self.n_templates

    # unwhiten template waveforms on their channels of max amplitude
    #if sparse.cols:
    #    raise NotImplementedError
    # apply the inverse whitening matrix to the template
    templates_wfs = np.zeros_like(sparse.data)  # nt, ns, nc
    for n in np.arange(n_wav):
        templates_wfs[n, :, :] = np.matmul(sparse.data[n, :, :], self.wmi)

    # The amplitude on each channel is the positive peak minus the negative
    templates_ch_amps = np.max(templates_wfs, axis=1) - np.min(templates_wfs, axis=1)

    # The template arbitrary unit amplitude is the amplitude of its largest channel
    # (but see below for true tempAmps)
    templates_amps_au = np.max(templates_ch_amps, axis=1)
    spike_amps = templates_amps_au[spikes] * self.amplitudes

    with np.errstate(divide='ignore', invalid='ignore'):
        # take the average spike amplitude per template
        templates_amps_v = (np.bincount(spikes, weights=spike_amps) /
                            np.bincount(spikes))
        # scale back the template according to the spikes units
        templates_physical_unit = templates_wfs * (templates_amps_v / templates_amps_au
                                                   )[:, np.newaxis, np.newaxis]

    return (spike_amps * sample2unit,
            templates_physical_unit * sample2unit,
            templates_amps_v * sample2unit)

In [51]:
m.get_amplitudes_true = get_amplitudes_true